In [1]:
# 导入所需的库
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import os
from sklearn.metrics import mean_squared_error

In [2]:
raw_train_data = pd.read_csv(r"C:\Users\yubin\Desktop\综合课程设计3\锂电池\train_data.csv")
raw_test_data = pd.read_csv(r"C:\Users\yubin\Desktop\综合课程设计3\锂电池\test_data.csv")
raw_dev_data = pd.read_csv(r"C:\Users\yubin\Desktop\综合课程设计3\锂电池\dev_data.csv")

In [26]:
# 删除第一列
train_data = raw_train_data.drop(raw_train_data.columns[0], axis=1)
dev_data = raw_dev_data.drop(raw_dev_data.columns[0], axis=1)
test_data = raw_test_data.drop(raw_test_data.columns[0], axis=1)
# 将值为0的数替换为NaN
train_data.replace(0, np.nan, inplace=True)

#填补缺失值
train_data['CCCT'] = train_data['CCCT'].fillna(train_data['CCCT'].mean())
train_data['CVCT'] = train_data['CVCT'].fillna(train_data['CVCT'].mean())

t_data = test_data[test_data['SoH'] == -100]


In [37]:
# 建立一个空的 DataFrame 来存储结果
results_df = pd.DataFrame(columns=['cycle', 'CS_Name', 'result'])

# 根据 CS_Name 列分组
# for group_name, group_data in train_data.groupby('CS_Name'):
#     # 划分特征和标签
#     X_group = group_data.drop(columns=['SoH'])
#     y_group = group_data['SoH']

# 数据预处理
# 可以根据需要进行特征选择、特征缩放、处理缺失值等操作

# 划分特征和标签
X_test = t_data.drop(columns=['SoH'])
# y_test = t_data['SoH']

# 建立空列表，用于存储每个组的模型
models = {}

# 根据 CS_Name 列分组
for group_name, group_data in train_data.groupby('CS_Name'):
    # 划分特征和标签
    y_group = group_data['SoH']
    group_data = group_data.drop(columns=['CS_Name'])
    group_data = group_data.drop(columns=['cycle'])
    group_data = group_data.drop(columns=['SoH'])


    # 建立XGB模型
    model = xgb.XGBRegressor(
        n_estimators=1000,  # 迭代次数
        max_depth=5,  # 树的最大深度
        learning_rate=0.1,  # 学习率
        objective='reg:squarederror',  # 损失函数
        random_state=42
    )

    # 训练模型
    model.fit(group_data, y_group)

    # 将训练好的模型存储到字典中
    models[group_name] = model

# 分组预测并计算RMSE
rmse_list = []

# 根据 CS_Name 列分组
for group_name, group_data_ in X_test.groupby('CS_Name'):
    # 获取对应组的模型
    model = models[group_name]
    group_data_=group_data_.drop(columns=['CS_Name'])
    group_data_=group_data_.drop(columns=['cycle'])
    # 对每个组进行预测
    y_pred_group = model.predict(group_data_)


    # 获取该组对应的 cycle 列
    cycles = X_test[X_test['CS_Name'] == group_name]['cycle']

    # 将结果添加到结果 DataFrame 中
    group_results = pd.DataFrame({
        'cycle': cycles,
        'CS_Name': [group_name] * len(cycles),
        'result': y_pred_group
    })

    results_df = pd.concat([results_df, group_results], ignore_index=True)

# 根据 'Cycle' 和 'CS_Name' 列进行内连接
merged_df = pd.merge(results_df, raw_test_data[[ 'Unnamed: 0','cycle', 'CS_Name']],
                     on=['cycle', 'CS_Name'], how='inner')
# 删除 merged_df 中的 'Unnamed: 0' 列，并插入到第一列位置
unnamed_column = merged_df.pop('Unnamed: 0')
merged_df.insert(0, unnamed_column.name, unnamed_column)
# 将合并后的结果保存为新的 CSV 文件
merged_df.to_csv('merged_file7.csv', index=False)


,cycle,CS_Name,result,Cycle,Result
0,735.0,CS2_35,0.644878,NaN,NaN
1,736.0,CS2_35,0.644878,NaN,NaN
2,737.0,CS2_35,0.644878,NaN,NaN
3,738.0,CS2_35,0.644878,NaN,NaN
4,739.0,CS2_35,0.644878,NaN,NaN
...,...,...,...,...,...
595,NaN,CS2_38,NaN,857.0,0.648464
596,NaN,CS2_38,NaN,858.0,0.648464
597,NaN,CS2_38,NaN,859.0,0.648464
598,NaN,CS2_38,NaN,860.0,0.648464


,Cycle,CS_Name,Result
545,832.0,CS2_38,0.648464
546,833.0,CS2_38,0.648676
547,834.0,CS2_38,0.647977
548,835.0,CS2_38,0.648464
549,836.0,CS2_38,0.648464
550,837.0,CS2_38,0.648464
551,838.0,CS2_38,0.648464
552,839.0,CS2_38,0.648464
553,840.0,CS2_38,0.648464
554,841.0,CS2_38,0.648464


0      0.825175
1      0.815965
2      0.815977
3      0.825194
4      0.806900
         ...   
624    0.641868
625    0.632691
626    0.632718
627    0.632666
628    0.623486
Name: SoH, Length: 629, dtype: float64

,capacity,resistance,CCCT,CVCT
1972,1.126964,0.102498,6585.419024,2359.168730
1973,1.126642,0.104031,6572.075291,2361.918249
1974,1.126019,0.104654,6566.714960,2362.605960
1975,1.124355,0.104031,6560.136514,2381.011933
1976,1.123409,0.101592,6612.742838,2194.316027
...,...,...,...,...
2676,0.824944,0.102405,4585.724740,2746.025763
2677,0.824955,0.099967,4600.053278,2731.026463
2678,0.824958,0.100148,4540.941728,2865.785299
2679,0.824926,0.100057,4459.971291,2847.144702
